# Joining DSD approvals data with SD county assessor parcel data

### Damon Crockett for Civic Knowledge, Inc.

#### damon@civicknowledge.com

In [1]:
import pandas as pd
from pandas.io import sql
from sqlalchemy import create_engine

In [2]:
engine = create_engine('sqlite:////Users/damoncrockett/Dropbox/thinkpad_desktop_2/assessor-db/parcels.geodb')

In [30]:
dd = sql.read_sql("SELECT * FROM parcels", engine)

In [9]:
len(dd.index)

1038477

In [11]:
dd.columns

Index([u'OGC_FID', u'GEOMETRY', u'apn', u'apn_8', u'parcelid', u'own_name1', u'own_name2', u'own_name3', u'fractint', u'own_addr1', u'own_addr2', u'own_addr3', u'own_addr4', u'own_zip', u'situs_juri', u'situs_stre', u'situs_suff', u'situs_post', u'situs_pre_', u'situs_addr', u'situs_frac', u'situs_buil', u'situs_suit', u'situs_comm', u'legldesc', u'asr_land', u'asr_impr', u'asr_total', u'doctype', u'docnmbr', u'docdate', u'acreage', u'taxstat', u'ownerocc', u'tranum', u'asr_zone', u'nucleus_zo', u'asr_landus', u'nucleus_us', u'situs_zip', u'submap', u'subname', u'unitqty', u'x_coord', u'y_coord', u'multi', u'overlay_ju', u'sub_type', u'addrno', u'addrfrac', u'addrunit', u'roadpdir', u'roadname', u'roadsfx', u'zip', u'shape_area', u'shape_len'], dtype='object')

In [31]:
dd = dd.iloc[:,2:]

In [42]:
dd = dd[['apn',
         'asr_land',
         'asr_impr',
         'asr_total',
         'acreage',
         'ownerocc',
         'tranum',
         'nucleus_zo',
         'asr_landus',
         'nucleus_us',
         'unitqty',
         'overlay_ju',
         'shape_area']]

In [44]:
len(dd.apn.value_counts())

1037451

In [49]:
dd = dd[dd.apn.notnull()]

In [50]:
len(dd.index)

1037451

In [51]:
dd.set_index('apn',inplace=True)

## approvals table

In [22]:
d = pd.read_csv('/Users/damoncrockett/vosd.org/215-opendsd/approval_master_FULL.csv')

/Library/Python/2.7/site-packages/pandas/io/parsers.py:1164: DtypeWarning: Columns (11,14) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [64]:
d.columns

Index([u'Unnamed: 0', u'APN', u'ApplicationDate', u'ApplicationExpiration', u'ApplicationExpired', u'ApplicationMonth', u'ApplicationYear', u'ApprovalId', u'Description', u'DevelopmentId', u'DevelopmentTitle', u'IssueDate', u'IssueMonth', u'IssueYear', u'IssuedBy', u'JobId', u'Latitude', u'Longitude', u'PermitHolder', u'ProjectId', u'ProjectManager', u'ProjectManagerId', u'ProjectScope', u'Scope', u'Status', u'StreetAddress', u'TimetoIssue', u'Title', u'Type', u'category'], dtype='object')

In [68]:
d = d[d.APN.notnull()]

In [71]:
l = pd.DataFrame(d.APN.str.split('-',3).tolist(),
                columns = ['a','b','c','d'])

In [72]:
d.APN = l.a+l.b+l.c+l.d

In [74]:
dd.rename(columns={'apn':'APN'},inplace=True)

In [75]:
d = d.join(dd,on='APN')

In [76]:
d.columns

Index([u'Unnamed: 0', u'APN', u'ApplicationDate', u'ApplicationExpiration', u'ApplicationExpired', u'ApplicationMonth', u'ApplicationYear', u'ApprovalId', u'Description', u'DevelopmentId', u'DevelopmentTitle', u'IssueDate', u'IssueMonth', u'IssueYear', u'IssuedBy', u'JobId', u'Latitude', u'Longitude', u'PermitHolder', u'ProjectId', u'ProjectManager', u'ProjectManagerId', u'ProjectScope', u'Scope', u'Status', u'StreetAddress', u'TimetoIssue', u'Title', u'Type', u'category', u'asr_land', u'asr_impr', u'asr_total', u'acreage', u'ownerocc', u'tranum', u'nucleus_zo', u'asr_landus', u'nucleus_us', u'unitqty', u'overlay_ju', u'shape_area'], dtype='object')

In [77]:
d = d[['ApplicationYear',
       'TimetoIssue',
       'Latitude',
       'Longitude',
       'Status',
       'StreetAddress',
       'Type',
       'category',
       'asr_land',
       'asr_impr',
       'asr_total',
       'acreage',
       'ownerocc',
       'tranum',
       'nucleus_zo',
       'asr_landus',
       'nucleus_us',
       'unitqty',
       'overlay_ju',
       'shape_area']]

In [84]:
d.head()

,ApplicationYear,TimetoIssue,Latitude,Longitude,Status,StreetAddress,Type,category,asr_land,asr_impr,asr_total,acreage,ownerocc,tranum,nucleus_zo,asr_landus,nucleus_us,unitqty,overlay_ju,shape_area
0,2009,999,32.727100,-117.100900,Cancelled - Selected Approval in Error,2286 FAIRMOUNT AV,Grading + Right of Way Permit,BOTH,1400000,600000,2000000,5.5,None,08241,60,36,360,1,SD,239583.72411
1,2005,287,32.792299,-117.241771,Issued,4026 RIVIERA DR,Coastal Development Permit,Discretionary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2005,287,32.792299,-117.241771,Issued,4026 RIVIERA DR,Tentative Map,Discretionary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2005,287,32.792570,-117.241852,Issued,4040 RIVIERA DR,Coastal Development Permit,Discretionary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2005,287,32.792570,-117.241852,Issued,4040 RIVIERA DR,Tentative Map,Discretionary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
d = d[d.asr_total.notnull()]

In [87]:
d.head()

,ApplicationYear,TimetoIssue,Latitude,Longitude,Status,StreetAddress,Type,category,asr_land,asr_impr,asr_total,acreage,ownerocc,tranum,nucleus_zo,asr_landus,nucleus_us,unitqty,overlay_ju,shape_area
0,2009,999,32.727100,-117.100900,Cancelled - Selected Approval in Error,2286 FAIRMOUNT AV,Grading + Right of Way Permit,BOTH,1400000,600000,2000000,5.50,None,08241,60,36,360,1,SD,239583.724110
18,2003,1802,32.849126,-117.270014,Issued,1327 COAST WK [Pending],Construction Change - Eng.,Ministerial,63394,76908,140302,5.46,None,08228,80,11,110,1,SD,244740.430660
20,2008,99,32.733793,-117.212729,Issued,2021 TRUXTUN RD,Construction Change - Eng.,Ministerial,63394,76908,140302,5.46,None,08228,80,11,110,1,SD,244740.430660
21,2008,226,32.733793,-117.212729,Issued,2021 TRUXTUN RD,Storm Water Maintenance Agrmnt,Ministerial,63394,76908,140302,5.46,None,08228,80,11,110,1,SD,244740.430660
22,2003,581,32.743009,-117.234217,Issued,4135 VOLTAIRE ST,Site Development Permit,Discretionary,7320248,17777747,25097995,0.84,None,08001,50,28,280,1,SD,36624.263554


In [88]:
d.to_csv('/Users/damoncrockett/vosd.org/215-opendsd/approvals_parcels.csv',encoding='utf-8')